In [40]:
#disclaimer: ChatGPT assisted with syntax only, logic and visualization ideas are original

import pandas as pd
import numpy as np
import altair as alt

alt.data_transformers.disable_max_rows()
df = pd.read_csv(r"C:\Users\Colin Briggs\Downloads\lego_filtered.csv")

#get top 10 themeGroups
top10 = df["themeGroup"].value_counts().nlargest(10).index.tolist()
df_top = df[df["themeGroup"].isin(top10)].copy()

#aggregate median max
agg = (
    df_top.groupby(["themeGroup", "year"])
          .agg(
              median_price=("US_retailPrice", "median"),
              max_price=("US_retailPrice", "max")
          )
          .reset_index()
)

#reshape
agg_long = agg.melt(
    id_vars=["themeGroup", "year"],
    value_vars=["median_price", "max_price"],
    var_name="metric",
    value_name="price"
)

#compute trendlines
year0 = 2000

def fit_line(group):
    x = group["year"] - year0
    y = group["price"]
    if len(group) < 2 or x.var() == 0:
        return pd.Series({"b0": np.nan, "b1": np.nan})
    b1, b0 = np.polyfit(x, y, 1)
    return pd.Series({"b0": b0, "b1": b1})

coefs = (
    agg_long.groupby(["themeGroup", "metric"])
            .apply(fit_line)
            .reset_index()
)

coefs["equation"] = (
    "y = "
    + coefs["b0"].round(2).astype(str)
    + " + "
    + coefs["b1"].round(2).astype(str)
    + " × (Year - "
    + str(year0)
    + ")"
)

#merge trend
agg_long = agg_long.merge(coefs, on=["themeGroup", "metric"], how="left")
agg_long["trend"] = agg_long["b0"] + agg_long["b1"] * (agg_long["year"] - year0)

#dropdown selection
theme_dropdown = alt.binding_select(options=top10, name="Theme Group: ")
theme_select = alt.selection_point(
    fields=["themeGroup"],
    bind=theme_dropdown,
    value=[{"themeGroup": top10[0]}]
)

#base chart
base = (
    alt.Chart(agg_long)
    .add_params(theme_select)
    .transform_filter(theme_select)
    .properties(width=800, height=350)
)

#price lines
lines = (
    base
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("price:Q", title="Price (USD)"),
        color=alt.Color("metric:N", title="Metric"),
        tooltip=[
            alt.Tooltip("themeGroup:N", title="Theme Group"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("metric:N", title="Metric"),
            alt.Tooltip("price:Q", title="Price in USD")
        ]
    )
)

#trendlines
trends = (
    base
    .mark_line(strokeDash=[4, 2])
    .encode(
        x=alt.X("year:O"),
        y=alt.Y("trend:Q"),
        color=alt.Color("metric:N", legend=None),
        tooltip=[
            alt.Tooltip("themeGroup:N", title="Theme Group"),
            alt.Tooltip("metric:N", title="Metric"),
            alt.Tooltip("trend:Q", title="Trend Value")
        ]
    )
)

#equation text (one row per metric under the chart)
eq_chart = (
    alt.Chart(coefs)
    .add_params(theme_select)
    .transform_filter(theme_select)
    .mark_text(align="left", dx=5)
    .encode(
        y=alt.Y("metric:N", title=None),
        text="equation:N"
    )
    .properties(width=800, height=60, title="Trendline Equations")
)

chart = (lines + trends) & eq_chart
chart


C:\Users\Colin Briggs\AppData\Local\Temp\ipykernel_8624\1104456732.py:45: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fit_line)
C:\Users\Colin Briggs\AppData\Local\Temp\ipykernel_8624\1104456732.py:125: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  chart = (lines + trends) & eq_chart


alt.VConcatChart(...)